<a href="https://colab.research.google.com/github/mauriciodev/spatialgeodesy/blob/main/exercises/Ex2_Broadcast_orbits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2 - Compute satellite coordinates from the navigation file

In this exercise we are going to download a navigation RINEX file, open it with the Georinex library and compute the coordinates of one satellite at a given time. The files are obtained from BKG, one of the International GNSS Service (IGS) data centers.

# Preparation

## Installing dependencies

In [ ]:
#Python standard libraries
import os #File path operations.
import shutil #Shell operations. Unzipping, moving files, etc.
import urllib.request #Downloader.

#External libraries
import numpy as np #Numeric Python.
import pandas as pd #Python Data Analysis Library.
import matplotlib.pyplot as plt #Plots.
import xarray as xa #Multi dimension arrays. For georinex.

In [ ]:
#Installing the package that reads rinex and sp3 files
#!pip install --upgrade pip
!pip install git+https://github.com/geospace-code/georinex
#brute force fix for a bug in keplerian.py
!sed -i -e 's/sv\["sv"\] in {\"R\", \"S\"}/sv\["sv"\] in ("R", "S")/g' /usr/local/lib/python3.10/dist-packages/georinex/keplerian.py
import georinex

# Input parameters

In [ ]:
#This is the reference time. We are going to use it to download the navigation files and compute the orbital position.
t = pd.to_datetime('2023-01-01 02:00:01')

In [ ]:
#CHANGE THIS: Use the code from the previous exercises to compute
year = t.year
dayOfYear = t.day_of_year
twoDigitYear = t.year %100

#File name is based on the Day of Year
#formatiing with previous variables. Note the :03 on each variable to fill with leading zeros.
#https://igs.bkg.bund.de/root_ftp/IGS/BRDC/2022/001/brdc0010.22n.gz
navigation_file_url=f"https://igs.bkg.bund.de/root_ftp/IGS/BRDC/{year}/{dayOfYear:03}/brdc{dayOfYear:03}0.{twoDigitYear}n.gz"
print(navigation_file_url)

## Downloading the broadcast Navigation File

In [ ]:
#splitting the file name from the URL
navRinexFile=os.path.split(navigation_file_url)[1]
local_filename, headers = urllib.request.urlretrieve(navigation_file_url, navRinexFile) #download the file saving as the name in zipFile
print(local_filename)

# Showing the first 20 lines of the navigation file

In [ ]:
# the ! prefix runs the commands on a linux shell. So we can use both python and shell commands on this notebook.
!uncompress $navRinexFile -k
uncompressed=navRinexFile[:-3]
!head -n 20  $uncompressed

# Opening the navigation file with Georinex

In [ ]:
nav = georinex.rinexnav(navRinexFile)
#Let's see the data that was read
nav

# Selecting subsets of the navigation file
Here we are selecting the Spatial Vehicle G01 (GPS), at the time previously specified in t. The method "pad" brings the "latest" orbit. Check the time of the orbit that we found on the "Coordinates" section. The orbital parameters are in the "Data variables".

In [ ]:
sat='G01'
orbit=nav.sel(sv=sat,time=t, method="pad")
orbit

# Implement the orbit calculation.
Now follow the formulas on the slides to implement the orbit calculation using the information of G01 at the given time t. Note that t is not the beginning of the orbit period and Toe (time of ephemeris) is counted in seconds of the GPS week.

In [ ]:
from numpy import sin, cos, arctan, sqrt #useful math functions
#CHANGE THIS: orbit calculation is unfinished. If you
earthrate = 7.2921151467e-5 #This is default bOMEGAE84
bGM84 = 3.986005e14 #GM

A=orbit.sqrtA**2 #the nav file provided the square root instead of A

print("A:",A.to_numpy())

print("Satellite position: ",satX,satY,satZ)

In [ ]:
#GPS time of week. You can use your own function from last exercise, if you prefer.
gps_t0=pd.to_datetime('1980-01-06 00:00:00')
deltaDate=t - gps_t0
dayOfWeek= deltaDate.days % 7
t_midnight=t.replace(hour=0, minute=0, second=0)
secondsOfGPSWeek=(t-t_midnight).seconds+dayOfWeek*24*60*60
print("t in seconds of GPS week", secondsOfGPSWeek, "TOE:", orbit.Toe.to_numpy())

# Orbit calculation with Georinex's function

In [ ]:
orbit['time']=[t] #We are now chaning the "time" dimension so that the orbit is calculated on a different time.
orbit

In [ ]:
#Check the code on the link to understand a bit how georinex compute the keplerian orbit
#https://github.com/geospace-code/georinex/blob/main/src/georinex/keplerian.py
x, y, z = georinex.keplerian2ecef(orbit)
print(x,y,z)

# How far are your coordinates from Georinex's orbit?